In [9]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from gensim.models import Word2Vec
from scipy.stats import mode
import matplotlib.pyplot as plt
import requests
from PIL import Image
from io import BytesIO
import warnings
import gensim
import pickle
from scipy.io import mmread
warnings.filterwarnings(action='ignore')

In [10]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
import pickle
from scipy.io import mmwrite, mmread
from sklearn.metrics.pairwise import linear_kernel
from tqdm import tqdm
import re
import numpy as np

In [11]:
df_origin = pd.read_csv('./data/beer_only_eng.csv', index_col = 0)
df_origin

ID               beer_name               date  grade  \
0       italianjohn(5,276)  Kloud Original Gravity  September 6, 2021    2.8   
1                orda(284)  Kloud Original Gravity    August 29, 2021    3.9   
2      Radek Kliber(8,065)  Kloud Original Gravity       June 5, 2021    2.4   
3        geologyguy(2,575)  Kloud Original Gravity       May 26, 2021    2.3   
4             Wolff90(203)  Kloud Original Gravity  February 16, 2021    1.0   
...                    ...                     ...                ...    ...   
83186          bourne(161)                   Cafri     March 21, 2006    1.3   
83187       curvedbeer(11)                   Cafri   November 6, 2005    1.0   
83188  tomciccateri(1,033)                   Cafri  September 5, 2007    2.2   
83189       Papsoe(25,039)                   Cafri      June 13, 2005    2.1   
83190      MrRomero(1,976)                   Cafri     March 21, 2004    1.7   

       Aroma  Appearance  Flavor  Mouthfeel  Overall  \
0        5.0         3.0     5.0        3.0     12.0   
1       10.0         5.0    10.0        5.0      9.0   
2        6.0         3.0     4.0        2.0      9.0   
3        4.0         3.0     4.0        3.0      9.0   
4        1.0         1.0     1.0        1.0      6.0   
...      ...         ...     ...        ...      ...   
83186    2.0         2.0     3.0        2.0      4.0   
83187    2.0         3.0     1.0        1.0      3.0   
83188    3.0         3.0     4.0        2.0     10.0   
83189    5.0         2.0     4.0        2.0      8.0   
83190    3.0         2.0     3.0        2.0      7.0   

                                             beer_review lang  
0      Clear golden blonde pour with a thin white hea...   en  
1      Appearance: golden amber; slight head\nTaste: ...   en  
2      Bottle 330 ml \n\nAroma: nice, fresh, on dry s...   en  
3      Smell is light of malt...taste is a that malt ...   en  
4      Rating copied from Untappd. Had while flying A...   en  
...                                                  ...  ...  
83186  I didnt think this was beer when I bought it. ...   en  
83187  Of all of the commercially brewed beers I’ve t...   en  
83188  From a can onboard a KAL flight. 4.2%, with no...   en  
83189  (Bottle 48 cl) Again - for Korea - this beauti...   en  
83190  Probably my favorite Korean beer but not as av...   en  

[72310 rows x 11 columns]

In [12]:
df_origin[df_origin['beer_name'] == 'Carlsberg Pilsner']

ID          beer_name                date  grade  \
37592       Arbitrage411(162)  Carlsberg Pilsner    October 24, 2022    2.9   
37593           Itstasty(263)  Carlsberg Pilsner  September 22, 2022    2.8   
37598              MacNJ(378)  Carlsberg Pilsner   February 28, 2022    1.0   
37605              JDsz5(614)  Carlsberg Pilsner      August 7, 2021    2.3   
37606  TrappistExports38(266)  Carlsberg Pilsner      August 2, 2021    1.5   
...                       ...                ...                 ...    ...   
39370              Deimos(81)  Carlsberg Pilsner   February 26, 2001    3.1   
39371           Oakes(28,107)  Carlsberg Pilsner    January 17, 2001    2.7   
39373           Gusler(2,653)  Carlsberg Pilsner    January 19, 2003    2.8   
39374             Sammys(427)  Carlsberg Pilsner   September 7, 2000    2.4   
39375             Bov(14,983)  Carlsberg Pilsner       April 4, 2007    1.9   

       Aroma  Appearance  Flavor  Mouthfeel  Overall  \
37592    4.0         3.0     5.0        3.0     14.0   
37593    5.0         3.0     6.0        3.0     11.0   
37598    3.0         3.0     1.0        1.0      2.0   
37605    5.0         2.0     5.0        2.0      9.0   
37606    3.0         2.0     3.0        2.0      5.0   
...      ...         ...     ...        ...      ...   
39370    5.0         3.0     6.0        3.0     14.0   
39371    5.0         2.0     6.0        3.0     11.0   
39373    6.0         3.0     6.0        3.0     10.0   
39374    4.0         3.0     5.0        3.0      9.0   
39375    3.0         2.0     5.0        2.0      7.0   

                                             beer_review lang  
37592  Gentle off white head forms over a pale gold i...   en  
37593  Bottle.\nNo great flavours, but on a red hot d...   en  
37598  Poured from a tall can into a Pilsner glass. V...   en  
37605  Aroma 5. Appearance 2. Taste 5. Palate 2. Over...   en  
37606  0.45l bottle from Chisinau. Light gold colour ...   en  
...                                                  ...  ...  
39370  Finally, a Euro Import that's not skunked.  Go...   en  
39371  Clean, malt-accented. It seems all Danish lage...   en  
39373  The first time I sampled this beer was on Octo...   en  
39374  "Probably the best beer in the world?" Nah.  A...   en  
39375  canned clear and pale golden colour, little co...   en  

[1549 rows x 11 columns]

In [13]:
df_review = pd.read_csv('./data/beer_CBF.csv', index_col = 0)
df_review

beer  \
0                  7Bräu Gompyo Wheatbeer   
1                        ARK Seoulite Ale   
2                         Asahi Super Dry   
3                   Asahi Super Dry Black   
4                  Bavaria 8.6 (Original)   
..                                    ...   
82                               Tsingtao   
83                    Tsingtao Wheat Beer   
84                        Victoria Bitter   
85  Volfas Engelman Grünberger Hefeweizen   
86               Warsteiner Premium Verum   

                                               review  
0   pours yellow loose lived head aroma peach flav...  
1   darker amber yet creamy foam creamy foam caram...  
2   quickly disintegrating head sweet malty notes ...  
3   copper beer tan coloured head toasty malt arom...  
4   head candy aroma corn candy alcohol bought mos...  
..                                                ...  
82  pours gold fizzy lived head lacing aroma malt ...  
83  banana clove aromas pale hint turbidities swee...  
84  works well degrees neutrals world coloured fai...  
85  pale yellow colour dense head lasts minutes cl...  
86  half liter tap cos oktoberfest pours color med...  

[87 rows x 2 columns]

In [14]:
df_review[df_review.beer=='Hite Beer']['review']

37    hite first ever barbecue houston food beer bes...
Name: review, dtype: object

In [15]:
df_origin.beer_review.iloc[:10]

0    Clear golden blonde pour with a thin white hea...
1    Appearance: golden amber; slight head\nTaste: ...
2    Bottle 330 ml \n\nAroma: nice, fresh, on dry s...
3    Smell is light of malt...taste is a that malt ...
4    Rating copied from Untappd. Had while flying A...
5    Strong hoppy flavor. Bavarian style. Ok but no...
6    Poured from 330mL bottle. Clear gold with whit...
7    Bottle. Tastes like dough, corn, grass, smooth...
8    Pours with a brilliant clear gold body and a m...
9    Small can from a supermarket in Phnom Penh. Po...
Name: beer_review, dtype: object

In [16]:
doc = []
for sentence in tqdm(df_origin.beer_review):
    sentences = re.sub(r"[^a-zA-Z0-9]"," ", sentence) # 한글이 아닌 문자는 공백으로 표시
    sentences = sentences.lower()
    new_string = sentences.replace('  ',' ')
    doc.append([new_string])

100%|█████████████████████████████████████████████████████████████████████████| 72310/72310 [00:00<00:00, 82594.88it/s]


In [17]:
doc2 = np.array(doc)
doc2.shape

(72310, 1)

In [18]:
df_origin['clean'] = doc2

In [19]:
df_origin['clean']

0        clear golden blonde pour with a thin white hea...
1        appearance golden amber slight head taste dece...
2        bottle 330 ml  aroma nice fresh on dry side yo...
3        smell is light of malt  taste is a that malt t...
4        rating copied from untappd had while flying as...
                               ...                        
83186    i didnt think this was beer when i bought it  ...
83187    of all of the commercially brewed beers i ve t...
83188    from a can onboard a kal flight 4 2  with no h...
83189     bottle 48 cl again  for korea  this beautiful...
83190    probably my favorite korean beer but not as av...
Name: clean, Length: 72310, dtype: object

## TFIDF 모델 생성

In [20]:
Tfidf = TfidfVectorizer(sublinear_tf=True,ngram_range=(1,2))
Tfidf_matrix = Tfidf.fit_transform(
    df_origin['clean'])
with open('tfidf.pickle', 'wb') as f:
    pickle.dump(Tfidf, f)
mmwrite('tfidf.mtx', Tfidf_matrix)

## Word2Vec 모델 생성

In [21]:
review_words = pd.read_csv(
    "C:/Users/82104/Documents/Sessac/자연어처리/가상환경/1st_project/전처리/preprocess_fin_fin.csv",
    index_col=0)
review_words.dropna(inplace = True)
clean_token_review = list(review_words['cleaned_review'])

In [22]:
import pandas as pd
from gensim.models import Word2Vec

# 파일 불러오기
review_words = pd.read_csv(
    "./data/beer_CBF.csv", index_col=0)
review_words.dropna(inplace = True)
clean_token_review = list(review_words['review'])

cleaned_tokens = []

# 토큰별로 변환
for sentence in clean_token_review:
    token = sentence.split(' ')
    cleaned_tokens.append(token)
print(len(cleaned_tokens))
print(len(sentence))
# 모델링 및 파라미터 조정
embedding_model = Word2Vec(cleaned_tokens,   # 벡터 사이즈 -> 차원 수
                          window=4, min_count=20,ns_exponent=0.75,
                           # ns_exponent -> 1.0 빈도에 정확히 비례하여 표본 추출,
                           # 0.0 값은 모든 단어를 동일하게 표본 추출하며 
                           # 음수 값은 고빈도 단어보다 저빈도 단어를 표본 추출
                          workers=-1,sg=1)
embedding_model.save('word2VecModel.model')

87
211561


In [23]:
df_review_one_sentence = pd.read_csv(
    "C:/Users/82104/Documents/Sessac/자연어처리/가상환경/1st_project/전처리/preprocess_fin_fin.csv", index_col=0)
df_review_one_sentence.dropna(inplace = True)
Tfidf = TfidfVectorizer(sublinear_tf=True,ngram_range=(1,2))
Tfidf_matrix = Tfidf.fit_transform(
    df_review_one_sentence['cleaned_review'])
# embedding_model = Word2Vec.load('word2VecModel_fin_fin.model')
# with open('Word2vec_final.pickle', 'wb') as f:
#     pickle.dump(embedding_model, f)
# with open('tfidf.pickle', 'wb') as f:
#     pickle.dump(Tfidf, f)
# mmwrite('tfidf.mtx', Tfidf_matrix)
df_review_one_sentence

title                                     cleaned_review
2            닌자 거북이  낙서 듯이 그린 디자인 시작 거북이 코믹 인기 합작 출판 인기 애니메이션 시리즈 제...
3            닌자 거북이  추억 닌자거북이 정겨운 거북이 복장 닌자터틀 이름 시즌 진짜 이름 만화가 일본 만화...
4            닌자 거북이                                레오나르도 라파엘 레더 닌자 거북이
5            닌자 거북이  슈퍼마리오 좋아했던 캐릭터 닌자거북이 닌자 호카게 설치는 수염 꼬맹이 유명하지만 닌...
6            닌자 거북이  뮤턴트 닌자터틀 주제곡 터틀 파워 에이 뮤턴트 닌자 터틀 닌자 거북이 닌자터틀 추억...
...             ...                                                ...
10283  히트맨: 에이전트 47  예정 히트 에이전트 이전 히트 티모시 올리펀트 주연 기억 좋게 사진 포스트 기억 주...
10284  히트맨: 에이전트 47  삼성동 메가박스 코엑스 히트 시리즈 링크 히트 히트 살인 면허 히트 에이전트 삼성동...
10285  히트맨: 에이전트 47  한편 여자 대박 흥행 순위 구심 히트 에이전트 기대 대물 기분 시놉 간단히 살인 병...
10286  히트맨: 에이전트 47  중계 액션 화끈하다 섹시하다 깔끔하다 정갈하다 아름답다 좋다 진짜 알차다 알차 치외...
10287  히트맨: 에이전트 47  아드레날린 폭발 인텔리전트 액션 에이전트 맞이 유전공학 통해 에이전트 클론 뛰어난 ...

[9824 rows x 2 columns]

In [24]:
abc = pd.read_csv("C:/Users/82104/Documents/Sessac/자연어처리/가상환경/1st_project/data/movie_review_2017_2020.csv", index_col= 0)
abc.iloc[0]

title                       100% 울프: 푸들이 될 순 없어 (100% Wolf)
review    크리스마스 아주 반가운 소식 울프 없어 최초 온라인 후원 상영 대작 온라인 가장 먼...
Name: 0, dtype: object

In [34]:
df_review_one_sentence = pd.read_csv(
    './data/beer_CBF.csv', index_col=0)
def getRecommendation(cosine_sim):
    simScores = list(enumerate(cosine_sim[-1]))
    simScores = sorted(simScores, key=lambda x:x[1],
                       reverse=True)
    simScores = simScores[:10]
    movieidx = [i[0] for i in simScores]
    RecMovielist = df_review_one_sentence.iloc[movieidx]
    return RecMovielist.beer

# movie_idx = df_review_one_sentence[
#     df_review_one_sentence[
#         'title']=='기생충 (PARASITE)'].index[0]
#movie_idx = 127
#print(df_review_one_sentence.iloc[movie_idx,0])
embedding_model = Word2Vec.load('./model/word2VecModel.model')
key_word = 'lager'
sentence = [key_word] * 10
if key_word in embedding_model.wv.vocab:
    sim_word = embedding_model.wv.most_similar(key_word, topn=10)
    #print(sim_word)
    labels = []
    for label, _ in sim_word:
        labels.append(label)
    print(labels)

    for i, word in enumerate(labels):
        sentence += [word] * (9-i)

sentence = ' '.join(sentence)
print(sentence)

# Tfidf = TfidfVectorizer(sublinear_tf=True)
# Tfidf_matrix = Tfidf.fit_transform(
#     df_review_one_sentence['review'])

with open("./model/tfidf.pickle", "rb") as fr:
    Tfidf = pickle.load(fr)
Tfidf_matrix = mmread("./model/tfidf.mtx")

sentence_vec = Tfidf.transform([sentence])
cosine_sim = linear_kernel(sentence_vec,
                           Tfidf_matrix)

recommendation = getRecommendation(cosine_sim)
print(recommendation)

['form', 'mild', 'mar', 'esthers', 'sensation', 'create', 'wonderful', 'beans', 'accompanied', 'evidence']
lager lager lager lager lager lager lager lager lager lager form form form form form form form form form mild mild mild mild mild mild mild mild mar mar mar mar mar mar mar esthers esthers esthers esthers esthers esthers sensation sensation sensation sensation sensation create create create create wonderful wonderful wonderful beans beans accompanied
51              Kloud Fitz Super Clear
76         Stephans Bräu Premium Lager
28    Gambrinus Originál 10° (Desítka)
23               Eichbaum Apostel Bräu
72                      Sapporo Yebisu
38               Hite Black Beer Stout
56                    Lapin Kulta IV A
41                      Hite Prime Max
63               Peroni Nastro Azzurro
29               Grolsch Premium Lager
Name: beer, dtype: object
